<a href="https://colab.research.google.com/github/angiellanos/MNIST_DF/blob/main/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Se usan las funciones *_load_label* y *_load_img* para cargar el conjunto de datos del MNIST. Luego, se usa la función *train_test_split* de la librería *sklearn* para dividir el conjunto de datos en un conjunto de entrenamiento y uno de prueba, asegurándose de que ambos estén balanceados. Es decir, que tengan la misma proporción de imágenes para cada dígito. Se usa el parámetro *stratify* de la función *train_test_split* para lograr esto. El tamaño del conjunto de prueba es 10000, entonces el conjunto de prueba tiene 10000 imágenes y el conjunto de entrenamiento 50000 imágenes.

In [54]:
# ! git clone https://github.com/angiellanos/MNIST_DF.git

In [55]:
%cd MNIST_DF/

/content/MNIST_DF/MNIST_DF


In [56]:
%ls

MNIST_DF/    t10k-images-idx3-ubyte.gz   train-labels-idx1-ubyte.gz
MNIST.ipynb  t10k-labels-idx1-ubyte.gz
README.md    train-images-idx3-ubyte.gz


In [57]:
print(os.getcwd())

/content/MNIST_DF/MNIST_DF


In [58]:
import os.path
import gzip
import pickle
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

files = {
    'train_img':'train-images-idx3-ubyte.gz',
    'train_label':'train-labels-idx1-ubyte.gz',
    'test_img':'t10k-images-idx3-ubyte.gz',
    'test_label':'t10k-labels-idx1-ubyte.gz'
}

image_size = 28
num_images = [60000, 10000]
num_labels = [60000, 10000]

def _load_label(file_key):
    """Función que carga las etiquetas de las imágenes desde un archivo comprimido en formato gzip.
    Args:
        file_name (str): el nombre del archivo que contiene las etiquetas
    Returns:
        array: etiquetas de las imágenes.
    """
    file_name = files[file_key]
    file_path = os.getcwd() + "/" + file_name

    j = 0 if file_key=='train_label' else 1
    with gzip.open(file_path, 'rb') as f:
        # omite los primeros 8 bytes
        f.read(8)
        buf = f.read(num_labels[j])
        label = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
        # print(label[:20])

    return label

def _load_img(file_key):
    """Función que carga las imágenes desde un archivo comprimido en formato gzip.
    Args:
        file_name (str): el nombre del archivo que contiene las etiquetas
    Returns:
        array: imágenes en forma de vectores de tamaño 784.
    """
    file_name = files[file_key]
    file_path = os.getcwd() + "/" + file_name

    j = 0 if file_key=='train_img' else 1
    with gzip.open(file_path, 'rb') as f:
        # omite los primeros 16 bytes
        f.read(16)
        # cada pixel en 1 byte = 8 bits
        # lee todos los datos y los coloca en un buffer de memoria
        buf = f.read(image_size * image_size * num_images[j])
        # traslada los datos a un array de numpy de tipo float32
        data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
        # Cambia la forma de los datos par entregarlos listos al dataset
        data = data.reshape(num_images[j], image_size, image_size, 1)

    return data

def _convert_numpy():
    """Función que convierte los datos del MNIST en arrays de NumPy y los guarda en dos diccionarios.
    Args: None
    Returns:
        dictionary: con las claves ‘train_img’, ‘train_label’, ‘test_img’ y ‘test_label’,
                   y los valores correspondientes a los arrays de NumPy con las imágenes
                   y las etiquetas.
    """
    # Cargar el conjunto de datos del MNIST usando las funciones _load_label y _load_img
    X_train = _load_img('train_img')
    y_train = _load_label('train_label')
    X_test = _load_img('test_img')
    y_test = _load_label('test_label')

    # Imprimir las dimensiones de los conjuntos de datos
    print("Dimensiones del conjunto de entrenamiento:")
    print(X_train.shape)
    print(y_train.shape)
    print("Dimensiones del conjunto de prueba:")
    print(X_test.shape)
    print(y_test.shape)

    return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = _convert_numpy()



Dimensiones del conjunto de entrenamiento:
(60000, 28, 28, 1)
(60000,)
Dimensiones del conjunto de prueba:
(10000, 28, 28, 1)
(10000,)


Una vez construidos los conjuntos de entrenamiento y prueba, se procede a construir el modelo de Naive Bayes. Para ello, se necesita calcular las probabilidades a priori de cada clase (P(c), donde c es la clase de cada dígito, p.e. la clase del 0, la del 1, hasta la del 9) y las probabilidades condicionales de cada píxel dado cada clase (P(x|c)). Estas probabilidades se pueden estimar a partir de los datos de entrenamiento usando la regla de Laplace, que consiste en añadir un pequeño valor (por ejemplo, 1) al numerador y al denominador de las frecuencias relativas.

Por lo tanto, la probabilidad a priori de cada clase $α$ (P(c=$α$)) está dada por:

In [59]:
# Contar el número total de imágenes
n_total = len(y_train)

# Crear una lista vacía para guardar las probabilidades a priori
p_c = []
n_c = []
# Recorrer las clases del 0 al 9
for c in range(10):
    # Contar el número de imágenes que tienen la etiqueta c
    n_c.append(np.sum(y_train == c))
    # Aplicar la regla de Laplace con un valor k=1
    p_c.append((n_c[c] + 1) / (n_total + 10))

# Convertir la lista en un array de numpy
p_c = np.array(p_c)
n_c = np.array(n_c)

print("P(C=c)=",p_c)

P(C=c)= [0.09871688 0.11236461 0.09930012 0.10218297 0.09736711 0.09035161
 0.09863356 0.10441593 0.09751708 0.09915014]


Estas probabilidades siempre serán cercanas a 0.1, que es la probabilidad esperada de cada dígito, dada por:

### $P(C=c)=\frac{Frecuencia \ del \ dígito}{Cantidad \ de \ dígitos}=\frac{1}{10}=0.1$

Debido a que son complementarias, su suma debe ser igual a 1. Verificando:

In [60]:
print("Suma de las probabilidades a priori:", np.sum(p_c))

Suma de las probabilidades a priori: 1.0


Para calcular las probabilidades condicionales de cada píxel dado cada clase (P(x|c)), se necesita contar el número de veces que cada píxel tiene un valor mayor que cero en las imágenes de cada clase. Así, se obtiene que:

In [61]:
# Crear una lista vacía para guardar las probabilidades condicionales
p_x_c = []
# Recorrer las clases del 0 al 9
for c in range(10):
    # Seleccionar las imágenes que tienen la etiqueta c
    X_c = X_train[np.where(y_train == c)]
    # Crear una lista vacía para guardar las probabilidades condicionales de cada píxel dado la clase c
    p_x_c_c = []
    # Recorrer los píxeles del 1 al 784
    for i in range(784):
        # Calcular el índice de la fila y la columna del píxel i
        row = i // 28
        col = i % 28
        # Contar el número de imágenes que tienen el píxel i mayor o igual que cero
        n_xi_c = np.sum(X_c[:, row, col, 0] > 0.1)
        # Aplicar la regla de Laplace con un valor k=1
        p_xi_c = (n_xi_c + 1) / (n_c[c] + 2)
        # Añadir la probabilidad condicional a la lista
        p_x_c_c.append(p_xi_c)
    # Convertir la lista en un array de numpy y añadirlo a la lista principal
    p_x_c.append(np.array(p_x_c_c))

# Convertir la lista principal en un array de numpy
p_x_c = np.array(p_x_c)


Note que la suma de las probabilidades condicionales por clase no tiene que ser 1, porque son independientes entre sí. Es decir, los pixeles se toman independientes entre sí para cada dígito en la imagen.

In [62]:
x_clase = np.sum(p_x_c, axis=1)
print("Suma de las probabilidades condicionales por clase:", x_clase)

Suma de las probabilidades condicionales por clase: [192.04236287  85.93683274 168.8840604  163.411218   141.89253936
 152.4182187  157.01435811 131.48045317 173.39039809 143.11645102]
